In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras as K
import GPy
import GPyOpt

from GPyOpt.methods import BayesianOptimization

# Importing dataset
from tensorflow.keras.datasets import mnist

# Getting featurees and labels
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()

# Flatten data
m, h, w = X_train.shape
mv, _, _ = X_test.shape
X_train = X_train.reshape(m, h * w)
X_test = X_test.reshape(mv, h * w)

# Data Normalization
X_train = X_train / 255
X_test = X_test / 255

# Labels Encoding
Y_train_oh = K.utils.to_categorical(Y_train)
Y_test_oh = K.utils.to_categorical(Y_test)

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 88 from C header, got 96 from PyObject
  return f(*args, **kwds)


In [2]:
def create_model(lr, beta1, beta2, l2_rate):
    """Creates a model
    """
    regularizer = K.regularizers.l2(l2_rate)
    inputs = K.layers.Input(shape=(784,))

    first = K.layers.Dense(
        150,
        activation='relu',
        kernel_regularizer=regularizer,
        name="first"
    )(inputs)
    second = K.layers.Dense(
        60,
        activation='relu',
        kernel_regularizer=regularizer,
        name="second"
    )(first)
    third = K.layers.Dense(
        120,
        activation='relu',
        kernel_regularizer=regularizer,
        name="third"
    )(second)
    outputs = K.layers.Dense(
        10,
        activation='softmax',
        name="outputs"
    )(third)
    model = K.Model(inputs, outputs)

    adam = K.optimizers.Adam(
        lr=lr,
        beta_1=beta1,
        beta_2=beta2
    )
    model.compile(
        loss='categorical_crossentropy',
        optimizer=adam,
        metrics=['accuracy']
    )
    return model

In [3]:
def fit_model(params):
    """Fit a model"""
    model = create_model(params[0][0], 0.9, 0.9, 0.1, 1e-4)

    blackbox = model.fit(
        X_train,
        Y_train_oh,
        batch_size=100,
        epochs=5,
        validation_split=0.15,
        verbose=0
    )
    # accuracy = blackbox.history['val_acc'][-1]
    val_accuracy = np.min(blackbox.history["val_loss"])
    return val_accuracy

In [4]:
domain = [{"name": "lr", "type": "continuous", "domain": (0.01, 0.001)}]

In [5]:
my_bayesian_opt = BayesianOptimization(
    fit_model,
    domain=domain,
    model_type="GP",
    initial_design_numdata=1,
    acquisition_type="EI",
    maximize=False,
    verbosity=True
)

In [9]:
my_bayesian_opt.run_optimization(max_iter=29,
                                         report_file="report",
                                         evaluations_file="evaluation",
                                         models_file="models")

In [10]:
my_bayesian_opt.plot_acquisition()

ValueError: min() arg is an empty sequence

In [19]:
(np.linspace(1, 30, 30))

(1.0,
 2.0,
 3.0,
 4.0,
 5.0,
 6.0,
 7.0,
 8.0,
 9.0,
 10.0,
 11.0,
 12.0,
 13.0,
 14.0,
 15.0,
 16.0,
 17.0,
 18.0,
 19.0,
 20.0,
 21.0,
 22.0,
 23.0,
 24.0,
 25.0,
 26.0,
 27.0,
 28.0,
 29.0,
 30.0)

In [25]:
np.arange(9).tolist()

[0, 1, 2, 3, 4, 5, 6, 7, 8]

In [1]:
#!/usr/bin/env python3
"""Hyperparameter Tuning"""
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras as K
import GPy
import GPyOpt
from GPyOpt.methods import BayesianOptimization

# Importing dataset
from tensorflow.keras.datasets import mnist

# Getting featurees and labels
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()

# Flatten data
m, h, w = X_train.shape
mv, _, _ = X_test.shape
X_train = X_train.reshape(m, h * w)
X_test = X_test.reshape(mv, h * w)

# Data Normalization
X_train = X_train / 255
X_test = X_test / 255

# Labels Encoding
Y_train_oh = K.utils.to_categorical(Y_train)
Y_test_oh = K.utils.to_categorical(Y_test)


def create_model(lr, beta1, beta2, dropout_rate, l2_rate):
    regularizer = K.regularizers.l2(l2_rate)

    inputs = K.layers.Input(shape=(784,))
    first = K.layers.Dense(
        48,
        activation='relu',
        kernel_regularizer=regularizer
    )(inputs)
    second = K.layers.Dense(
        16,
        activation='relu',
        kernel_regularizer=regularizer
    )(first)
    third = K.layers.Dense(
        32,
        activation='relu',
        kernel_regularizer=regularizer
    )(second)
    dropout = K.layers.Dropout(
        dropout_rate
    )(third)
    outputs = K.layers.Dense(
        10,
        activation='softmax'
    )(dropout)
    model = K.Model(inputs, outputs)

    adam = K.optimizers.Adam(
        lr=lr,
        beta_1=beta1,
        beta_2=beta2
    )

    model.compile(
        loss='categorical_crossentropy',
        optimizer=adam,
        metrics=['accuracy']
    )
    return model


def fit_model(hyperprameters):
    """Fit a model"""
    lr = hyperprameters[0][0]
    beta1 = hyperprameters[0][1]
    beta2 = hyperprameters[0][2]
    dropout_rate = hyperprameters[0][3]
    l2_rate = hyperprameters[0][4]
    batch_size = hyperprameters[0][5]
    epochs = hyperprameters[0][6]

    model = create_model(lr, beta1, beta2, dropout_rate, l2_rate)

    blackbox = model.fit(
        X_train,
        Y_train_oh,
        batch_size=batch_size,
        epochs=epochs,
        validation_split=0.15,
    )
    accuracy = blackbox.history['val_acc'][-1]
    return accuracy


lr_domain = np.logspace(-9, 0, 10)
beta1_domain = np.logspace(-9, 0, 10)
beta2_domain = np.logspace(-9, 0, 10)
dropout_domain = np.linspace(0, 0.9, 10)
l2_domain = np.logspace(-9, 0, 10)
batch_domain = np.linspace(10, 100, 10)
epochs_domain = np.linspace(1, 30, 30)

bounds = [
    {"name": "lr", "type": "continuous", "domain": tuple(lr_domain)},
    {"name": "beta1", "type": "continuous", "domain": tuple(beta1_domain)},
    {"name": "beta2", "type": "continuous", "domain": tuple(beta2_domain)},
    {"name": "dropout_rate", "type": "continuous", "domain": tuple(dropout_domain)},
    {"name": "l2_rate", "type": "continuous", "domain": tuple(l2_domain)},
    {"name": "batch_size", "type": "continuous", "domain": tuple(batch_domain)},
    {"name": "epochs", "type": "continuous", "domain": tuple(epochs_domain)},
]
b_optimization = BayesianOptimization(
    fit_model,
    domain=bounds,
    model_type="GP",
    initial_design_numdata=1,
    acquisition_type="EI",
    maximize=False,
    verbosity=True
)

b_optimization.run_optimization(max_iter=30)


/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 88 from C header, got 96 from PyObject
  return f(*args, **kwds)
Train on 51000 samples, validate on 9000 samples


TypeError: 'numpy.float64' object cannot be interpreted as an integer

In [2]:
1e-1

0.1

In [3]:
np.linspace(0, 0.9, 10)


array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])

In [5]:
np.logspace(0, 9, 10)

array([1.e+00, 1.e+01, 1.e+02, 1.e+03, 1.e+04, 1.e+05, 1.e+06, 1.e+07,
       1.e+08, 1.e+09])

In [8]:
np.linspace(10, 100, 10)

array([ 10.,  20.,  30.,  40.,  50.,  60.,  70.,  80.,  90., 100.])

In [2]:
def create_model(lr, beta1, beta2, dropout_rate, l2_rate):
    regularizer = K.regularizers.l2(l2_rate)

    inputs = K.layers.Input(shape=(784,))
    first = K.layers.Dense(
        48,
        activation='relu',
        kernel_regularizer=regularizer
    )(inputs)
    second = K.layers.Dense(
        16,
        activation='relu',
        kernel_regularizer=regularizer
    )(first)
    third = K.layers.Dense(
        32,
        activation='relu',
        kernel_regularizer=regularizer
    )(second)
    dropout = K.layers.Dropout(
        dropout_rate
    )(third)
    outputs = K.layers.Dense(
        10,
        activation='softmax'
    )(dropout)
    model = K.Model(inputs, outputs)

    adam = K.optimizers.Adam(
        lr=lr,
        beta_1=beta1,
        beta_2=beta2
    )

    model.compile(
        loss='categorical_crossentropy',
        optimizer=adam,
        metrics=['accuracy']
    )
    return model

In [3]:
model = create_model(.1, 0.9, .99, .2, .1)

blackbox = model.fit(
    X_train,
    Y_train_oh,
    batch_size=128,
    epochs=10,
    validation_split=0.15,
)
accuracy = blackbox.history['val_acc'][-1]

Train on 51000 samples, validate on 9000 samples
Epoch 1/10
51000/51000 [==============================] - 3s 63us/step - loss: 3.9615 - acc: 0.1096 - val_loss: 2.4471 - val_acc: 0.1093
Epoch 2/10
51000/51000 [==============================] - 3s 53us/step - loss: 2.4244 - acc: 0.1049 - val_loss: 2.4257 - val_acc: 0.0959
Epoch 3/10
51000/51000 [==============================] - 3s 62us/step - loss: 2.4105 - acc: 0.1066 - val_loss: 2.4060 - val_acc: 0.1063
Epoch 4/10
51000/51000 [==============================] - 3s 57us/step - loss: 2.4079 - acc: 0.1066 - val_loss: 2.4086 - val_acc: 0.1063
Epoch 5/10
51000/51000 [==============================] - 3s 55us/step - loss: 2.4074 - acc: 0.1081 - val_loss: 2.4109 - val_acc: 0.1063
Epoch 6/10
51000/51000 [==============================] - 3s 58us/step - loss: 2.4082 - acc: 0.1021 - val_loss: 2.4190 - val_acc: 0.1093
Epoch 7/10
51000/51000 [==============================] - 3s 55us/step - loss: 2.4078 - acc: 0.1050 - val_loss: 2.4062 - val_acc: